This largely comes from the NEON tutorial  https://www.neonscience.org/calc-biomass-py

In [7]:
import numpy as np
import pandas as pd
import os
import gdal, osr
import matplotlib.pyplot as plt
import sys
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
%matplotlib inline 

#Import biomass specific libraries
from skimage.morphology import watershed
from skimage.feature import peak_local_max
from skimage.measure import regionprops
from sklearn.ensemble import RandomForestRegressor

In [2]:
#Define plot band array function

def plot_band_array(band_array,image_extent,title,cmap_title,colormap,colormap_limits):
    plt.imshow(band_array,extent=image_extent)
    cbar = plt.colorbar(); plt.set_cmap(colormap); plt.clim(colormap_limits)
    cbar.set_label(cmap_title,rotation=270,labelpad=20)
    plt.title(title); ax = plt.gca()
    ax.ticklabel_format(useOffset=False, style='plain') 
    rotatexlabels = plt.setp(ax.get_xticklabels(),rotation=90)

In [3]:
def array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array,epsg):

    cols = array.shape[1]
    rows = array.shape[0]
    originX = rasterOrigin[0]
    originY = rasterOrigin[1]

    driver = gdal.GetDriverByName('GTiff')
    outRaster = driver.Create(newRasterfn, cols, rows, 1, gdal.GDT_Float32)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    outband = outRaster.GetRasterBand(1)
    outband.WriteArray(array)
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromEPSG(epsg)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()

In [23]:
tiles = pd.read_csv('tile_list_BART_TEAK_HARV.tsv', sep=' ', header=None)
tiles.columns = ['tiles']
tiles = list(tiles.tiles)
chm_file = 'C:/RSDI_2017/Day4/Biomass/NEON_D17_SJER_DP3_256000_4106000_CHM.tif'

In [24]:
tiles

['NEON_D01_BART_DP3_316000_4880000',
 'NEON_D01_BART_DP3_315000_4879000',
 'NEON_D01_BART_DP3_315000_4880000',
 'NEON_D01_BART_DP3_318000_4879000',
 'NEON_D01_BART_DP3_317000_4879000',
 'NEON_D01_BART_DP3_318000_4881000',
 'NEON_D01_BART_DP3_318000_4880000',
 'NEON_D01_BART_DP3_317000_4878000',
 'NEON_D01_BART_DP3_316000_4882000',
 'NEON_D01_BART_DP3_317000_4880000',
 'NEON_D01_BART_DP3_314000_4881000',
 'NEON_D01_BART_DP3_316000_4879000',
 'NEON_D01_BART_DP3_314000_4880000',
 'NEON_D01_BART_DP3_316000_4881000',
 'NEON_D01_BART_DP3_317000_4881000',
 'NEON_D01_BART_DP3_314000_4879000']